In [35]:
import pandas as pd
import sqlalchemy
from routes import routes
from utils import graph_utils, station_manager, visualization_utils

In [36]:
fluxos = (pd.read_csv(r'../data/raw/ArqSIADEFluxoTransporteRealizado_03_02_2025.csv', sep='|', encoding='windows-1252', dtype={'NumeroCNPJ': str})
        .assign(DataReferencia=lambda df: pd.to_datetime(df['DataReferencia'], format='%m/%d/%Y %H:%M:%S').dt.strftime('%Y-%m'),
                id=lambda df: df['CodigoFluxoTransporteFerrovia'] + df['SiglaFerrovia'])
        .rename(columns={
            'DataReferencia': 'data',
            'SiglaFerrovia': 'ferrovia',
            'NomeMercadoria': 'mercadoria',
            'RazaoSocialClientePessoaJuridica': 'cliente',
            'CodigoTresLetrasEstacaoDe': 'origem',
            'CodigoTresLetrasEstacaoPara': 'destino',
            'ValorTU': 'TU',
            'DistanciaItinerario': 'dist_media',
            'CodigoFluxoTransporteFerrovia': 'cod_flux',
            'NumeroCNPJ': 'cnpj'
        })
        #.reindex(columns=['id','data', 'ferrovia', 'mercadoria', 'cliente', 'origem', 'destino', 'TU', 'dist_media', 'cod_flux', 'cnpj'])
        .pipe(lambda df: df[~df['data'].isnull()])
        .pipe(lambda df: df[df['dist_media'] > 0.000].reset_index(drop=True))
        )

/tmp/ipykernel_433713/3616010791.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  fluxos = (pd.read_csv(r'../data/raw/ArqSIADEFluxoTransporteRealizado_03_02_2025.csv', sep='|', encoding='windows-1252', dtype={'NumeroCNPJ': str})


In [37]:
fluxos['data'] = pd.to_datetime(fluxos['data'])
fluxos[fluxos['data']>='2024-11-01'][['ferrovia', 'cliente',
       'mercadoria', 
       'dist_media', 'origem',  'destino', 'data',
       'TU',  'ValorTKU', 'id']]

,ferrovia,cliente,mercadoria,dist_media,origem,destino,data,TU,ValorTKU,id
56,FTL,FERROVIA TRANSNORDESTINA LOGÍSTICA S.A,Demais Produtos,748.665,ATO,BNR,2024-11-01,965.0,722461.0,I0348FTL
79,FTL,FERROVIA TRANSNORDESTINA LOGÍSTICA S.A,Demais Produtos,59.318,BNR,BCK,2024-11-01,3.0,177.0,I0354FTL
486,MRS,FERRO METAIS SIDERURGIA LTDA.,Ferro Gusa,550.679,FZN,FAR,2024-11-01,1115.0,614007.0,32692MRS
496,MRS,FERRO METAIS SIDERURGIA LTDA.,Ferro Gusa,687.347,EPM,FAR,2024-11-01,4198.0,2885482.0,32693MRS
512,MRS,LGLO SIDERURGICA LTDA,Ferro Gusa,550.679,FZN,FAR,2024-11-01,1157.0,637135.0,32754MRS
...,...,...,...,...,...,...,...,...,...,...
491796,EFVM,FERTILIZANTES HERINGER S/A,Ureia,1413.295,VTU,EBJ,2024-12-01,957.0,1352523.0,K1372EFVM
491797,EFVM,FERTIGRAN FERTILIZANTES VALE DO RIO GRANDE LTDA,Outros - adubos e Fertilizantes,1413.295,VTU,EBJ,2024-12-01,1608.0,2272578.0,K1409EFVM
491798,EFVM,FERTIGRAN FERTILIZANTES VALE DO RIO GRANDE LTDA,Outros - adubos e Fertilizantes,1413.295,VTU,EBJ,2024-12-01,5525.0,7808454.0,K1413EFVM
491799,EFVM,FERTILIZANTES TOCANTINS S.A,Outros - adubos e Fertilizantes,1413.295,VTU,EBJ,2024-12-01,7987.0,11287987.0,K1448EFVM


In [38]:
# import stations
stations = routes.extracts('../data/processed/stations.gpkg')
remover = routes.select_criteria(stations) # filter stations (same criteria as routes)
stations = station_manager.remove_stations(stations, remover) # remover as estações deletadas; aqui tem que ser o mesmo critério de routes
# make graph
G = graph_utils.generate_graph(stations)

In [39]:
engine = sqlalchemy.create_engine('sqlite:///../data/processed/database.db')

def import_query(file_path):
    with open(file_path, 'r') as file:
        return file.read()

query = """SELECT 
    o.data,
    o.id,
    o.TU,
    o.ValorTKU,
    f.ferrovia,
    f.cliente,
    f.mercadoria,
    f.origem,
    f.destino,
    f.dist_media,
    f.intermed,
    f.len_Dijkstra,
    f.rota
FROM 
    ocorrencias o
LEFT JOIN 
    fluxos_rotas_2017 f
ON 
    o.id = f.id
WHERE 
    o.data >= '2021-01-01' and o.data <= '2025-12-01';
"""
fluxos = pd.read_sql(query, engine)

In [40]:
fluxos.sort_values(by='data')

,data,id,TU,ValorTKU,ferrovia,cliente,mercadoria,origem,destino,dist_media,intermed,len_Dijkstra,rota
0,2021-01-01 00:00:00.000000,10657RMS,27.0,16844.0,RMS,BRADO LOGISTICA S/A,Contêiner Vazio de 40 Pés,LND,LDP,623.868,LND LAP LUS LDP,623.868,LND LRL LAG LCV LAP LJM LCF LUV LGE LMF LOD LB...
1124,2021-01-01 00:00:00.000000,29855RMN,695.0,1147348.0,RMN,COFCO INTERNATIONAL GRAINS LTDA.,Grãos - Milho,TRO,ICZ,1650.861,TRO TMI ZTO ZOI ZIQ ZBV ZQB ZKE ZEV ZPG IAA IP...,1650.861,TRO TSF TAL TBS TSA TIQ TCT TMJ TEP TBE TAG TB...
1123,2021-01-01 00:00:00.000000,29856RMN,762.0,1257956.0,RMN,COFCO INTERNATIONAL GRAINS LTDA.,Grãos - Milho,TRO,ICZ,1650.861,TRO TMI ZTO ZOI ZIQ ZBV ZQB ZKE ZEV ZPG IAA IP...,1650.861,TRO TSF TAL TBS TSA TIQ TCT TMJ TEP TBE TAG TB...
1122,2021-01-01 00:00:00.000000,11171RMS,402.0,206495.0,RMS,PETROBRAS DISTRIBUIDORA S A,Álcool,LMN,LAW,513.670,LMN LAP LUS LAW,513.671,LMN LSD LMA LMI LJD LAI LPR LAP LJM LCF LUV LG...
1121,2021-01-01 00:00:00.000000,84469RMP,9703.0,7871908.0,RMP,USINA CORURIPE AÇÚCAR E ÁLCOOL S/A,Açúcar,ZFN,ISN,811.286,ZFN ZAR ZIQ ZBV ZQB ZKE ZEV ZPG ICB ISN,811.291,ZFN ZMR ZGI ZVP ZZM ZKY ZEC ZTM ZEB ZVU ZMO ZR...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
107777,2024-12-01 00:00:00.000000,27834RMN,5367.0,8839416.0,RMN,AMAGGI EXPORTACAO E IMPORTACAO LTDA,Grãos - Milho,TRO,ICZ,1646.994,TRO TMI ZTO ZOI ZIQ ZBV ZQB ZKE ZEV ZPG IAA IP...,1650.861,TRO TSF TAL TBS TSA TIQ TCT TMJ TEP TBE TAG TB...
107789,2024-12-01 00:00:00.000000,27796RMN,379.0,624210.0,RMN,BUNGE ALIMENTOS S/A,Grãos - Milho,TRO,ICZ,1646.994,TRO TMI ZTO ZOI ZIQ ZBV ZQB ZKE ZEV ZPG IAA IP...,1650.861,TRO TSF TAL TBS TSA TIQ TCT TMJ TEP TBE TAG TB...
109056,2025-01-01 00:00:00.000000,08468EFPO,9516.0,2289530.0,EFPO,COOPERATIVA AGRÁRIA AGROINDUSTRIAL,Soja,SCA,S01,240.598,SCA S01,240.598,SCA SCB SIB SGU SHE SLS SCG SGO SAR SLI SCR S01
109055,2025-01-01 00:00:00.000000,02011EFPO,7353.0,1769117.0,EFPO,COOPERATIVA AGRÁRIA AGROINDUSTRIAL,Soja,SCA,S01,240.598,SCA S01,240.598,SCA SCB SIB SGU SHE SLS SCG SGO SAR SLI SCR S01


In [48]:
fluxos['Diff'] = abs(fluxos['dist_media'] - fluxos['len_Dijkstra'])
round(fluxos['Diff'],0).unique()

array([ 4.,  3.,  0.,  2.,  6.,  1.,  5.,  8.,  9., 35., 27., 33., 26.])

Mercadorias:
- grãos = farelo, milho e soja
- açucar

Ano: 2021 a hoje

Trechos:
- Santos (ISN)   candidatas: (ICZ)
- Paranaguá (LDP)   candidatas: (LAT)
- São Francisco (LFC)   candidatas: (LAY e LJL)


In [42]:
# fluxo 
mercha = [ 'Açúcar', 'Farelo de Milho', 'Farelo de Soja', 'Farinha de Trigo', 'Grãos - Milho', 'Grãos - Trigo', 'Grãos - arroz', 'Soja']
[ 'Açúcar', 'Farelo de Milho', 'Farelo de Soja',  'Grãos - Milho', 'Soja']

['Açúcar', 'Farelo de Milho', 'Farelo de Soja', 'Grãos - Milho', 'Soja']

In [43]:
fluxos = fluxos[fluxos['mercadoria'].isin(mercha)].copy()

print(fluxos['destino'].unique())

# portos
santos = ['ISN','ICZ']
paranagua = ['LDP']
sao_fran_sul = ['LFC']

fluxos['santos'] = fluxos['destino'].isin(santos) * fluxos['TU']
fluxos['paranagua'] = fluxos['destino'].isin(paranagua) * fluxos['TU']
fluxos['sao_fran_sul'] = fluxos['destino'].isin(sao_fran_sul) * fluxos['TU']

['ICZ' 'IUF' 'ISN' 'LDP' 'NRG' 'LFC' 'QPI' 'ATZ' 'S01' 'VTU' 'LLD' 'LRL'
 'SCA' 'NCZ' 'LID' 'LBH' 'PPF' 'EBJ' 'QPM' 'PPT' 'NIJ' 'LMG' 'TRO' 'ZPD'
 'ATQ' 'NJC' 'LMA' 'VWI' 'ICB' 'NCY' 'BAR' 'EAU' 'QAL']


In [44]:
# coluna Ano
fluxos['data'] = pd.to_datetime(fluxos['data'])
fluxos['Ano'] = fluxos['data'].dt.year


In [45]:
fluxos['flag_acucar'] = fluxos['mercadoria']=='Açúcar'

In [46]:
# groupby 'Ano' e 'mercadoria'
fluxos_grouped = fluxos.groupby(['Ano', 'flag_acucar']).agg({'santos':'sum', 'paranagua':'sum', 'sao_fran_sul':'sum'}).reset_index()
fluxos_grouped.T

,0,1,2,3,4,5,6,7,8
Ano,2021,2021,2022,2022,2023,2023,2024,2024,2025
flag_acucar,False,True,False,True,False,True,False,True,False
santos,25242007.0,6759562.0,32365754.0,6038951.0,33957049.0,6130554.0,33409042.0,6815031.0,0.0
paranagua,5068877.0,3810817.0,5091841.0,3911287.0,5151058.0,4448028.0,5042569.0,4430019.0,0.0
sao_fran_sul,2073490.0,0.0,2744202.0,0.0,3509432.0,0.0,3583167.0,0.0,0.0


São Paulo: Porto de Santos - Estações:
Santos (ISN);
Conceiçãozinha (ICZ);
Estação TUF (IUF);
Cubatão (ICB).

Paraná: Porto de Paranaguá - Estações:
D Pedro II (LDP);
km 5 (LID).

Santa Catarina: Porto de São Francisco do Sul - Estação:
São Francisco do Sul (LFC).